In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
try:
    env = spark.conf.get("pipeline.env")
except:
    env = "dev"
catalog = "book_rec_catalog"
bronze_schema = f"{env}_bronze"
silver_schema = f"{env}_silver"
spark.sql(f"USE CATALOG {catalog}")
spark.sql(f"USE SCHEMA {silver_schema}")

In [0]:
# ————————————————————————————————————————————————
# FINAL NON-STREAMING TABULKY PRO STREAMING
# ————————————————————————————————————————————————

def promote_to_batch_table(table_name: str):
    """
    Funkce vytváří nové tabulky ze silver tabulek pro použití s Row Level Security a Column Masking.
    
    Args:
        table_name: Název silver tabulky k promoci
    """
    source_path = f"{catalog}.{silver_schema}.{table_name}"
    target_path = f"{catalog}.{silver_schema}.{table_name}_batch"
    
    print(f"Promoting {source_path} to {target_path}")
    
    try:
        (
            spark.read.table(source_path)
                 .write
                 .mode('overwrite')
                 .option('overwriteSchema', 'true')
                 .saveAsTable(target_path)
        )
        print(f"Successfully promoted {table_name} to {table_name}_batch")
    except Exception as e:
        print(f"Error promoting {table_name}: {str(e)}")

# Seznam tabulek pro promoci
tables_to_promote = [
    'books_silver',
    'ratings_silver', 
    'users_silver'
]

print(f"Starting promotion of {len(tables_to_promote)} tables...")
print("=" * 60)

for table in tables_to_promote:
    promote_to_batch_table(table)

print("=" * 60)
print("Promotion completed!")
